<a href="https://colab.research.google.com/github/AparnaMadva/Miscellaneous/blob/main/KannadaNER_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "transformers[torch]" datasets ipywidgets sentencepiece seqeval

In [ ]:
# import the tokenizer and model.
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
label_list = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

model = AutoModelForTokenClassification.from_pretrained("JKA-NLP/KannadaNER", num_labels=len(label_list))
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
tokenizer.config = model.config

In [ ]:
def get_predictions( sentence, tokenizer, model ):
  tok_sentence = tokenizer(sentence, return_tensors='pt')

  with torch.no_grad():
    logits = model(**tok_sentence).logits.argmax(-1)
    predicted_tokens_classes = [model.config.id2label[t.item()] for t in logits[0]]

    predicted_labels = []
    previous_token_id = 0
    # only label the first token of a given word.
    word_ids = tok_sentence.word_ids()
    for word_index in range(len(word_ids)):
        if word_ids[word_index] == None:
            previous_token_id = word_ids[word_index]
        elif word_ids[word_index] == previous_token_id:
            previous_token_id = word_ids[word_index]
        else:
            predicted_labels.append( predicted_tokens_classes[ word_index ] )
            previous_token_id = word_ids[word_index]

    return predicted_labels

In [ ]:
example = "ಅದನ್ನೇ ಇನ್ನೊಬ್ಬರು ವಿಜ್ಞಾನಿಗಳು ಅಮೇರಿಕದಲ್ಲಿ ಕೊತ್ತೋ ಅದೊಂದು ಪೋಸ್ಟಿಂಗ್ ಮಾಡಿದ್ದರೆ ನನ್ನ ಹೆಸರು ರಮೇಶ್ ತುುವೇಕೆರೆ"

In [ ]:
predicted_labels = get_predictions(sentence=example, tokenizer=tokenizer, model=model)

print(example.split(" "))
print(predicted_labels)

['ಅದನ್ನೇ', 'ಇನ್ನೊಬ್ಬರು', 'ವಿಜ್ಞಾನಿಗಳು', 'ಅಮೇರಿಕದಲ್ಲಿ', 'ಕೊತ್ತೋ', 'ಅದೊಂದು', 'ಪೋಸ್ಟಿಂಗ್', 'ಮಾಡಿದ್ದರೆ', 'ನನ್ನ', 'ಹೆಸರು', 'ರಮೇಶ್', 'ತುುವೇಕೆರೆ']
['O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER']
